In [1]:
import midiParser as mp
import mido
import sklearn
import importlib
importlib.reload(mp)

<module 'midiParser' from 'C:\\Users\\cjgra\\Documents\\GitHub\\Music-Generation\\MIDI\\midiParser.py'>

In [2]:
song = mido.MidiFile('./mary.mid')
corpus = mp.to_arr(song, note_only = True)

In [3]:
times = mp.to_arr(song, time_only =True)

In [4]:
def basic_bigram(corpus):
    bigram_freqs = {}
    bigrams = [(corpus[i], corpus[i+1]) for i in range(len(corpus)-1)]
    for bigram in bigrams:
        bigram_freqs[bigram] = bigram_freqs.get(bigram, 0) + 1
    
    return bigram_freqs

In [5]:
basic_bigram(corpus)

{(64, 48): 5,
 (48, 62): 2,
 (62, 55): 7,
 (55, 60): 3,
 (60, 52): 2,
 (52, 62): 2,
 (55, 64): 7,
 (48, 64): 2,
 (64, 55): 3,
 (64, 52): 2,
 (52, 55): 2,
 (55, 62): 3,
 (62, 47): 2,
 (47, 62): 2,
 (62, 53): 1,
 (53, 55): 1,
 (48, 67): 1,
 (67, 55): 1,
 (55, 67): 1,
 (67, 52): 1,
 (52, 64): 1,
 (64, 53): 1,
 (53, 62): 1,
 (60, 48): 1,
 (48, 55): 1,
 (55, 48): 1}

In [6]:
from nltk import bigrams
from collections import defaultdict
model = defaultdict(lambda: defaultdict(lambda: 0))

In [7]:
for n1,n2 in bigrams(corpus):
    model[n1][n2] += 1

In [8]:
for n1 in model:
    total_count = float(sum(model[n1].values()))
    for n2 in model[n1]:
        model[n1][n2] /= total_count

In [9]:
# check if notes exist
len(model[60])

2

In [10]:
import random
r = random.random()
y = .0

In [11]:
import random
starting_note = [60]
if len(model[starting_note[-1]]):
    song_finished = False
    while not song_finished:
        r = random.random()
        y = .0
        for note in model[starting_note[-1]]:
            y+= model[starting_note[-1]][note]
            if y>r:
                starting_note.append(note)
                break
        if len(starting_note) == 60:
            song_finished = True
    print(starting_note)
else:
    print('Starting note did not exist')

[60, 48, 55, 64, 48, 64, 48, 67, 55, 64, 48, 62, 55, 67, 52, 55, 60, 52, 62, 53, 55, 64, 52, 55, 64, 48, 55, 64, 55, 60, 48, 67, 55, 62, 55, 64, 48, 62, 47, 62, 55, 64, 48, 67, 55, 60, 48, 55, 60, 52, 55, 64, 55, 60, 48, 64, 48, 67, 52, 55]


In [12]:
mp.note_to_midi(starting_note, 'test_3')

In [13]:
song = mido.MidiFile('./mary.mid')
corpus = mp.to_arr(song)

In [14]:
import os
import pandas as pd
import sqlite3
query_list = []
for filename in os.listdir('data'):
    f = os.path.join('data',filename)
    for file in os.listdir(f):
        path = os.path.join(f,file)
        song = mido.MidiFile(path)
        query = [filename, file, mp.to_arr(song, velocity_based=True), mp.to_arr(song, time_only = True)]
        query[2] = list(map(str, query[2])) #changes to strings for use with nltk
        query[3] = list(map(str, query[3])) #changes to strings for use with nltk
        query_list.append(query)
song_data = pd.DataFrame(query_list, columns = ['composer','filename','note_corpus', 'time_corpus'])

In [15]:
song_data

,composer,filename,note_corpus,time_corpus
0,albeniz,alb_esp1.mid,"[81, 88, 86, 88, 86, 84, 86, 88, 89, 91, 88, 8...","[240, 240, 0, 1560, 0, 60, 0, 60, 0, 240, 0, 2..."
1,albeniz,alb_esp2.mid,"[57, 69, 66, 62, 57, 62, 66, 69, 57, 62, 66, 6...","[360, 120, 0, 0, 0, 480, 0, 0, 360, 120, 0, 0,..."
2,albeniz,alb_esp3.mid,"[71, 75, 78, 76, 78, 76, 72, 81, 71, 75, 78, 7...","[0, 120, 120, 120, 120, 120, 120, 80, 0, 80, 0..."
3,albeniz,alb_esp4.mid,"[79, 74, 75, 79, 78, 81, 79, 81, 79, 78, 79, 8...","[0, 120, 120, 120, 120, 120, 120, 120, 120, 12..."
4,albeniz,alb_esp5.mid,"[75, 79, 77, 74, 80, 77, 77, 74, 75, 79, 72, 7...","[1920, 0, 720, 0, 0, 0, 240, 0, 0, 0, 480, 0, ..."
...,...,...,...,...
290,tschai,ty_maerz.mid,"[67, 67, 69, 67, 66, 69, 67, 70, 74, 78, 79, 7...","[2160, 60, 60, 40, 0, 40, 0, 40, 0, 360, 0, 12..."
291,tschai,ty_mai.mid,"[62, 66, 74, 71, 62, 67, 71, 62, 69, 74, 71, 6...","[2100, 30, 30, 480, 0, 180, 30, 30, 0, 0, 0, 6..."
292,tschai,ty_november.mid,"[73, 71, 76, 68, 71, 80, 83, 85, 83, 80, 71, 6...","[0, 240, 0, 240, 0, 480, 0, 480, 0, 0, 0, 240,..."
293,tschai,ty_oktober.mid,"[65, 69, 77, 69, 67, 76, 65, 69, 74, 68, 69, 6...","[0, 0, 480, 480, 0, 0, 0, 480, 0, 480, 0, 0, 0..."


In [16]:
song_3 = mido.MidiFile('data/albeniz/alb_esp1.mid')
test_notes = mp.to_arr(song_3, velocity_based = True)
# Interesting, these midi files don't use notes on and off, they set the notes 
# velocity to 0
# this means my to_arr() fucntion won't work to properly create bigrams,
# as there are a bunch of extra notes - should be fixed mostly now

In [17]:
from nltk import bigrams
from collections import defaultdict
model_2 = defaultdict(lambda: defaultdict(lambda: 0))

In [18]:
large_corpus = song_data['note_corpus'].to_list()
time_corpus = song_data['time_corpus'].to_list()

In [56]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(4, large_corpus)
ttrain, tvocab = padded_everygram_pipeline(4, time_corpus)

In [57]:
from nltk.lm import MLE
lm = MLE(2)
lm2 = MLE(2)

In [58]:
lm.fit(train, vocab)
lm2.fit(ttrain, tvocab)

In [59]:
len(lm.vocab)

90

In [37]:
test_song = lm.generate(60, random_seed = 13)
test_time = lm2.generate(60, random_seed = 13)

In [38]:
test_song = list(map(int, test_song))
test_time = list(map(int, test_time))
# change back to int for easier convert to midi

In [ ]:
mp.note_to_midi(test_song, 'large_corpus')
mp.time_to_midi(test_song, test_time, 'notes_and_times')

#### 5-gram Model:

In [70]:
from nltk.lm import MLE
lm_5 = MLE(5)
lm2_5 = MLE(5)

In [71]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(5, large_corpus)
ttrain, tvocab = padded_everygram_pipeline(5, time_corpus)
# Not sure why I need to reinstate the generater object, should just restart by itself

In [72]:
lm_5.fit(train, vocab)
lm2_5.fit(ttrain, tvocab)

In [50]:
test_song = lm_5.generate(60, random_seed = 13)
test_time = lm2_5.generate(60, random_seed = 13)

In [53]:
test_song = list(map(int, test_song))
test_time = list(map(int, test_time))
# change back to int for easier convert to midi

In [54]:
mp.note_to_midi(test_song, '5gram')
mp.time_to_midi(test_song, test_time, '5gram_parallel')

In [75]:
test_song = lm_5.generate(120, random_seed = 13)
test_time = lm2_5.generate(120, random_seed = 13)
test_song = list(map(int, test_song))
test_time = list(map(int, test_time))

In [76]:
mp.note_to_midi(test_song, '5gram_long')
mp.time_to_midi(test_song, test_time, '5gram_parallel_long')

#### 15-gram model

In [63]:
from nltk.lm import MLE
lm_15 = MLE(15)
lm2_15 = MLE(15)

In [64]:
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(15, large_corpus)
ttrain, tvocab = padded_everygram_pipeline(15, time_corpus)
# Not sure why I need to reinstate the generater object, should just restart by itself

In [65]:
lm_15.fit(train, vocab)
lm2_15.fit(ttrain, tvocab)

In [66]:
test_song = lm_15.generate(200, random_seed = 13)
test_time = lm2_15.generate(200, random_seed = 13)
test_song = list(map(int, test_song))
test_time = list(map(int, test_time))
# change back to int for easier convert to midi

In [68]:
mp.note_to_midi(test_song, '15gram')
mp.time_to_midi(test_song, test_time, '15gram_parallel')